In [1]:

import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier

from preprocess import over_resample
from evaluate import evaluate_model, evaluate_all_models, predict_test
import pandas as pd
from tqdm import tqdm
import itertools
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
import pandas as pd
import numpy as np
from evaluate import evaluate_model, evaluate_all_models, predict_test
from models import model_class, feature_cols
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from cross_validation_test import cross_validation, custom_group_cv_with_smote
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


2025-05-23 16:00:55.760002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-23 16:00:55.822551: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-23 16:00:55.824402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-23 16:00:56.832472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sakana/anaconda3/envs/ML/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.

In [2]:
enhanced_df_raw = pd.read_csv('../python_data_pre/engineered_features_train_nor_s.csv')
# enhanced_df_raw = enhanced_df_raw[enhanced_df_raw['language_code'].isin([12,17,18])]
#
# enhanced_df_raw = enhanced_df_raw[~enhanced_df_raw['language_code'].isin([12])]

# Get all unique language codes
language_codes = enhanced_df_raw['language_code'].unique()
print(f"Found {len(language_codes)} unique languages: {language_codes}")

# Dictionary to store best models for each language
language_models = {}
language_models2 = {}

# Train best model for each language
for lang_code in language_codes:
    best_score_lang=0
    # Filter data for this language
    enhanced_df = enhanced_df_raw[enhanced_df_raw['language_code'] == lang_code]
    lang_name = enhanced_df['language'].iloc[0] if not enhanced_df.empty else f"Language {lang_code}"
    print(f"\n{'='*60}")
    print(f"Processing {lang_name} (code: {lang_code})")
    print(f"{'='*60}")

    if enhanced_df.empty or len(enhanced_df) < 50:
        print(f"Insufficient data for language code {lang_code}, skipping.")
        continue

    # Prepare features and target
    X = enhanced_df[feature_cols]
    y = enhanced_df['is_root']

    # Split data into training and testing sets
    groups = enhanced_df['sentence_id']
    gss = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    train_idx, test_idx = next(gss.split(X, y, groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    # Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled, X_test, y_test, train_idx, test_idx = over_resample(enhanced_df)

    # Perform cross-validation to find top 3 models
    print(f"Performing cross-validation for {lang_name}...")
    top3 = cross_validation(X_train, y_train, X_test, y_test, train_idx, test_idx, model_class, groups, enhanced_df)

    if not top3:
        print(f"No valid models found for {lang_name}")
        continue

    # Get the best model (first in top3)
    best_model_name = list(top3.keys())[0]
    best_model_info = top3[best_model_name]

    print(f"\nBest model for {lang_name}: {best_model_name}")
    print("-" * 50)

    # Create and train the best model
    model_class_test = best_model_info['model_class']
    model_params = best_model_info['best_params']
    best_model = model_class_test(**model_params)
    best_model.fit(X_train_resampled, y_train_resampled)


    language_models[lang_code] = {
        'name': lang_name,
        'model': best_model,
        'X_test': X_test,
        'y_test': y_test,
        'test_idx': test_idx,
        'df': enhanced_df
    }

    # best_model2=DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=1)
    # best_model2.fit(X_train_resampled, y_train_resampled)
    #
    # language_models2[lang_code] = {
    #     'name': lang_name,
    #     'model': best_model2,
    #     'X_test': X_test,
    #     'y_test': y_test,
    #     'test_idx': test_idx,
    #     'df': enhanced_df
    # }

    # except:
    #     print('Error occurred during processing')



Found 21 unique languages: [12  4  6  3  8  5 11 10 18 17  9  7 13 14 19 20  2  1 15  0 16]

Processing Japanese (code: 12)
Class distribution before resampling:
0    10536
1      425
Name: is_root, dtype: int64

Processing Finnish (code: 4)
Class distribution before resampling:
0    5341
1     425
Name: is_root, dtype: int64

Processing Galician (code: 6)
Class distribution before resampling:
0    8561
1     425
Name: is_root, dtype: int64

Processing English (code: 3)
Class distribution before resampling:
0    7560
1     425
Name: is_root, dtype: int64

Processing Hindi (code: 8)
Class distribution before resampling:
0    8812
1     425
Name: is_root, dtype: int64

Processing French (code: 5)
Class distribution before resampling:
0    9097
1     425
Name: is_root, dtype: int64

Processing Italian (code: 11)
Class distribution before resampling:
0    8756
1     425
Name: is_root, dtype: int64

Processing Indonesian (code: 10)
Class distribution before resampling:
0    6845
1     425
N

In [3]:
language_models=language_models2

In [4]:
estimators = [(f"model_{code}", model_data['model']) for code, model_data in language_models.items()]

ensemble = VotingClassifier(
    estimators=estimators,
    voting='soft'  # Use probability-based voting
)


In [5]:
X_combined = []
y_combined = []

for lang_code, model_data in language_models.items():
    # Get original dataframe for this language
    lang_df = model_data['df']

    # Extract features and target from the original dataframe
    # Assuming you have a function to extract features from the dataframe
    # For simplicity, we'll use the test data that's already processed
    X_lang = model_data['X_test']
    y_lang = model_data['y_test']

    # Add to combined dataset
    X_combined.append(X_lang)
    y_combined.append(y_lang)

# Concatenate all data
if X_combined and y_combined:
    X_ensemble = np.vstack(X_combined)
    y_ensemble = np.concatenate(y_combined)

    # Fit the ensemble
    print(f"Fitting ensemble with {len(X_ensemble)} samples...")
    ensemble.fit(X_ensemble, y_ensemble)

Fitting ensemble with 29887 samples...


In [6]:
for lang_code in language_models:
    print(lang_code)
    print(language_models[lang_code]['model'])
    # print(language_models2[lang_code]['model'])


12
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
4
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
6
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
3
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
8
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
5
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
11
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
10
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
18
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
17
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
9
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
7
DecisionTreeClassifier(class_weight='balanced', max_depth=1, random_state=42)
13
DecisionTreeClassifier(class_wei

In [7]:
ensemble

VotingClassifier(estimators=[('model_12',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_4',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_6',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_3',
                              DecisionTreeClassifier(class_weight='balanced...
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_15',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_0',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42)),
                             ('model_16',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     max_depth=1,
                                                     random_state=42))],
                 voting='soft')

In [8]:
test_df=pd.read_csv('../python_data_pre/engineered_features_test_nor_s.csv')
# test_df= test_df[~ test_df['language_code'].isin([12])]
test_result=predict_test(ensemble, test_df)
test=test_result[test_result['is_root']==1 ]
# test.to_csv('../python_data_after/test_result.csv')

[[0.80058114 0.19941886]
 [0.80058114 0.19941886]
 [0.27923431 0.72076569]
 ...
 [0.80058114 0.19941886]
 [0.27923431 0.72076569]
 [0.80058114 0.19941886]]


In [9]:
test=test_result[test_result['is_root']==1 ]
test.head()

,id,language,language_code,sentence_id,vertex,is_root
2,1,Japanese,12,1,10,1
43,2,Japanese,12,3,17,1
92,3,Japanese,12,4,15,1
130,4,Japanese,12,6,6,1
149,5,Japanese,12,7,3,1


In [10]:
import pandas as pd
submit=pd.read_csv('../python_data_pre/submit_result_1.csv')
submit=submit[submit['id'].isin(test['id'])]
print(len(test), len(submit))

10395 10395


In [11]:
submit.head()

,id,vertex
0,1,4
1,2,17
2,3,5
3,4,15
4,5,9


In [12]:
test.head()

,id,language,language_code,sentence_id,vertex,is_root
2,1,Japanese,12,1,10,1
43,2,Japanese,12,3,17,1
92,3,Japanese,12,4,15,1
130,4,Japanese,12,6,6,1
149,5,Japanese,12,7,3,1


In [13]:

c=0
for idx,row in submit.iterrows():
    if test[test['id']==row['id']]['vertex'].values[0] == row['vertex']:
        c=c+1

print(c/len(submit))

0.5808561808561808


In [14]:
test.loc[:,['id', 'vertex']].to_csv('../python_data_after/test_3.csv', index=False)